# Coherent dark states in a 3-level system
Investigating coherent dark states in 3-level system using the `toy_systems` package. 

The 3-level system will consist of two ground states which are both coupled to the same excited state, which in turn can decay back to both of the ground states. A similar situation can be found in TlF when driving optical transitions from the electronic ground state X to the excited state B. The spacing of hyperfine levels in X is much smaller than the natural linewidth of the transitions, so that multiple groundstate hyperfine states end up coupling to one excited state hyperfine state. The number of coherent dark states in this scenario is N-1 where N is the number of ground states coupling to the same excited state. The dark states are typically not eigenstates of the internal Hamiltonian (e.g. TlF internal Hamiltonian) and so they evolve into bright states over time - we'll investigate the rate at which this occurs as the coupling strength of the ground states to the excited state and their energy splitting is varied.
## Imports
Start by importing the necessary packages

In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
plt.style.use("ggplot")
import numpy as np
import qutip
from sympy import Symbol

from toy_systems.couplings import ToyCoupling, ToyEnergy
from toy_systems.dark_states import get_dark_states
from toy_systems.decays import ToyDecay
from toy_systems.hamiltonian import Hamiltonian
from toy_systems.quantum_system import QuantumSystem
from toy_systems.states import Basis, BasisState, ToyQuantumNumbers

## Set up states and basis
We start by defining the three states of the system: the ground states $|g0\rangle$ and $|g1\rangle$, and the excited state $|e\rangle$>

In [2]:
g0 = BasisState(qn=ToyQuantumNumbers(label="g0"))
g1 = BasisState(qn=ToyQuantumNumbers(label="g1"))
e = BasisState(qn=ToyQuantumNumbers(label="e"))

# Define basis
basis = Basis((g0,g1,e))
basis.print_basis()

|0> = |g0>
|1> = |g1>
|2> = |e>


## Define energies, couplings, decays and quantum system
I'm going to define the system in the rotating frame as usual.

### Energies

In [3]:
δ = Symbol('delta') # Energy splitting between |g0> and |g1
Δ = Symbol('Delta') # Detuning of drive field from 0
E0 = ToyEnergy(g0, -δ/2)
E1 = ToyEnergy(g1, +δ/2)
Ee = ToyEnergy(e, Δ)

### Couplings

In [4]:
Ω0 = Symbol('Omega_0') # Drive field Rabi rate for |g0> <-> |e>
Ω1 = Symbol('Omega_1') # Drive field Rabi rate for |g1> <-> |e>

coupling_0 = ToyCoupling(g0,e,Ω0)
coupling_1 = ToyCoupling(g1,e,Ω1)

### Decays
Defining a decay from $|e\rangle$ to $|g0\rangle$ and $|g1\rangle$:

In [5]:
decay_0 = ToyDecay(e, ground = g0, gamma = Symbol("Gamma_0"))
decay_1 = ToyDecay(e, ground = g1, gamma = Symbol("Gamma_1"))

### Define a QuantumSystem
The QuantumSystem object combines the basis, Hamiltonian and decays to make setting parameters for time evolution using QuTiP more convenient.

In [6]:
# Define the system
system = QuantumSystem(
    basis=basis,
    couplings=[E0, E1, Ee, coupling_0, coupling_1],
    decays=[decay_0, decay_1],
)

# Get representations of the Hamiltonian and the decays that will be accepted by qutip
Hqobj, c_qobj = system.get_qobjs()

## Find dark states
There is some rudimentary functionality in `toy_systems` for finding the bright and dark states when a number of ground states are coupled to a single excited state. Let's test that here:

In [12]:
bright_state, dark_states = get_dark_states([g0, g1], e, [coupling_0, coupling_1])

print(f"|B> =\n{bright_state}")

print(f"\n|D> =\n{dark_states[0]}")

print(f"\n<B|D> = {dark_states[0]@bright_state}")

|B> =
0.71+0.00j x |g0>
0.71+0.00j x |g1>

|D> =
0.71+0.00j x |g0>
-0.71+0.00j x |g1>

<B|D> = 0j


## Time evolution
No matter what state, the system starts in, it should always end up in the dark state, from which it will slowly decay out since the dark state is not an eigenstate of the Hamiltonian.

In [ ]:
# Generate a Qobj representing the initial state
psi0 = (1*g0).qobj(basis)

# Operators for getting probability of being in each state as a function of time
P_g0 = qutip.Qobj((1*g0).density_matrix(basis), type = "oper")
P_g1 = qutip.Qobj((1*g1).density_matrix(basis), type = "oper")
P_e = qutip.Qobj((1*e).density_matrix(basis), type = "oper")
P_B = qutip.Qobj(bright_state.density_matrix(basis), type = "oper")
P_D = qutip.Qobj(dark_states[0].density_matrix(basis), type = "oper")